## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [7]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Weather Description
0,0,Marang,5.2056,103.2059,81.10,78,59,1.52,MY,2022-04-18 00:03:39,broken clouds
1,1,Yellowknife,62.4560,-114.3525,19.92,49,20,8.05,CA,2022-04-17 23:59:02,few clouds
2,2,Panuco,22.0500,-98.1667,84.29,64,17,16.04,MX,2022-04-18 00:03:40,few clouds
3,3,Ushuaia,-54.8000,-68.3000,46.06,93,100,16.11,AR,2022-04-17 23:59:40,moderate rain
4,4,Castro,-24.7911,-50.0119,52.02,96,8,4.34,BR,2022-04-18 00:01:28,clear sky


In [8]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [9]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Weather Description
0,0,Marang,5.2056,103.2059,81.10,78,59,1.52,MY,2022-04-18 00:03:39,broken clouds
2,2,Panuco,22.0500,-98.1667,84.29,64,17,16.04,MX,2022-04-18 00:03:40,few clouds
10,10,Rikitea,-23.1203,-134.9692,78.55,63,3,11.43,PF,2022-04-18 00:00:51,clear sky
17,17,Ixtapa,20.7000,-105.2000,78.03,69,75,8.05,MX,2022-04-18 00:03:43,broken clouds
26,26,Hilo,19.7297,-155.0900,77.29,92,100,1.99,US,2022-04-18 00:00:41,mist
27,27,Vaini,-21.2000,-175.2000,80.98,79,99,13.78,TO,2022-04-18 00:01:16,overcast clouds
33,33,Kavieng,-2.5744,150.7967,85.57,66,83,14.18,PG,2022-04-18 00:03:47,broken clouds
34,34,Atuona,-9.8000,-139.0333,79.65,78,8,17.07,PF,2022-04-18 00:03:47,clear sky
35,35,Mahebourg,-20.4081,57.7000,75.25,87,36,11.41,MU,2022-04-18 00:01:29,light rain
40,40,Faanui,-16.4833,-151.7500,80.96,72,44,9.22,PF,2022-04-18 00:03:49,light rain


In [10]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                211
City                   211
Lat                    211
Lng                    211
Max Temp               211
Humidity               211
Cloudiness             211
Wind Speed             211
Country                211
Date                   211
Weather Description    211
dtype: int64

In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# no empty rows but would append .dropna()
# preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)].dropna()

In [14]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,Marang,MY,81.10,broken clouds,5.2056,103.2059,
2,Panuco,MX,84.29,few clouds,22.0500,-98.1667,
10,Rikitea,PF,78.55,clear sky,-23.1203,-134.9692,
17,Ixtapa,MX,78.03,broken clouds,20.7000,-105.2000,
26,Hilo,US,77.29,mist,19.7297,-155.0900,
27,Vaini,TO,80.98,overcast clouds,-21.2000,-175.2000,
33,Kavieng,PG,85.57,broken clouds,-2.5744,150.7967,
34,Atuona,PF,79.65,clear sky,-9.8000,-139.0333,
35,Mahebourg,MU,75.25,light rain,-20.4081,57.7000,
40,Faanui,PF,80.96,light rain,-16.4833,-151.7500,


In [18]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
   
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [22]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City                   211
Country                211
Max Temp               211
Weather Description    211
Lat                    211
Lng                    211
Hotel Name             211
dtype: int64

In [23]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [25]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Description</dt><dd>{Weather Description}</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [27]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))